In [2]:
# PACKAGES
import shutup
shutup.please()

# DATA MANIPULATION AND NUMERICAL ANALYSIS
import math
import pandas as pd                                                            # data analysis and manipulation tool
from pandas.api.types import is_numeric_dtype                                  # it checks if a pnadas column is numeric
import numpy as np                                                             # package for scientific computing

# MACHINE LEARNING
from sklearn.linear_model import LinearRegression                              # simple linear regression
from sklearn.model_selection import train_test_split                           # splits the data into a train and test data set
import statsmodels.api as sm                                                   # advanced regression
from sklearn.metrics import mean_squared_error                                 # root mean square error
from sklearn.linear_model import ElasticNet                                    # elastic net
from sklearn.linear_model import ElasticNetCV                                  # elastic net with cross validation
from sklearn.model_selection import GridSearchCV                               # parameter tuning
from sklearn.model_selection import RepeatedKFold                              # cross validation
from sklearn.metrics import fbeta_score, make_scorer                           # custom score function
from sklearn.preprocessing import StandardScaler

# VISUALIZATION
import matplotlib.pyplot as plt                                                # data visualization  
import seaborn as sns                                                          # data visualization lib. based on matplotlib
sns.set_theme()                                                                # seaborn theme

# SETTINGS
pd.set_option('display.max_columns', 1000)
csfont = {'fontname':'Arial'}             

# MAGIC FUNCTION 
%matplotlib inline

# WARNINGES
import warnings
warnings.filterwarnings('ignore')


In [3]:
# FUNCTIONS

# this functions transforms non-numeric values into a dummy coding:
def make_dummies(dat): 
    
    ''' note: this funciton does not consider ordinal variables '''
    
    df = pd.DataFrame()
    df.index = dat.index

    for col in dat.columns:

        if is_numeric_dtype(dat[col]):
            
            df = df.merge(
                dat[col].fillna(0), 
                left_index=True, 
                right_index=True
            )
        
        else:
            
            df = df.merge(
                pd.get_dummies(dat[col], drop_first=True, prefix = col, dtype=int), 
                left_index=True, 
                right_index=True
            )
            
    return df


# this function splits a data set into train and test data
def make_split(dt, target):
    X_train, X_test, y_train, y_test = train_test_split( 
        dt.drop([target],axis=1),                                       
        dt[target],                                                          
        test_size=0.3, 
        train_size=0.7
    )
    
    return X_train, X_test, y_train, y_test      

# this function calculates the root-mean-squared-error (RMSE)
def get_RMSE(y_true, y_pred):
    '''
    Metric: Root-Mean-Squared-Error (RMSE) between the log. of the predicted value and 
    the logarithm of the observed sales price.
    '''
    y_trug_log = np.log(np.abs(y_true))
    y_pred_log = np.log(np.abs(y_pred))
    
    return math.sqrt(mean_squared_error(y_trug_log, y_pred_log))

In [34]:
# DATA LOAD
train = pd.read_csv('train.csv')                                                  # loads the train data     
test  = pd.read_csv('test.csv')                                                   # loads the test data

train.set_index('Id', inplace=True)                                               # setting index 'ID'
test.set_index('Id', inplace=True)                                                # setting index 'ID'

#train = make_dummies(train)                                                       # feature engineering - creating dummies for non-num. val.
#X_train, X_test, y_train, y_test = make_split(train, 'SalePrice')                 # splits the data into a train and a test data set

#y_train = pd.DataFrame(np.log(y_train), index=y_train.index)
#y_test = pd.DataFrame(np.log(y_test), index=y_test.index)


# scaling test over all features
#sc = StandardScaler()
#saved_cols = X_train.columns
#X_train_sc = sc.fit_transform(X_train)
#X_test_sc = sc.transform(X_test)

#X_train = pd.DataFrame(X_train_sc, columns=saved_cols, index=X_train.index)
#X_test  = pd.DataFrame(X_test_sc, columns=saved_cols, index=X_test.index)

In [35]:
train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [36]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [33]:
train['LotShape_IR2'].unique()

array([0, 1])

In [31]:
# columns with numerical elements
dt_num = []                                    

# columns with ordinal elements
dt_ord = []

# columns with nominal elements
dt_nom = []

# columns with boolean elements 0/1 coded                                  
dt_boo = [
    
]                                    